## virus host composition (that actually infect) + family histogram 

In [53]:
import pandas as pd
import numpy as np
import plotly.express as px
from itables import init_notebook_mode, show


df = pd.read_json("dataset_sanitise/galiez2017/taxonomy_fixing/server/virus.json", orient="index")
expanded_df = df['host'].apply(lambda x: pd.Series(x['lineage_names']))
expanded_df.columns = [f"host_{taxlevel}" for taxlevel in ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']]
df = pd.concat([df.drop('host', axis=1), expanded_df], axis=1)
#count occcurences in the host_family column
show(df)
counts = df['host_species'].value_counts().reset_index()
show(counts)
host_counts = df[[*expanded_df.columns]].drop_duplicates()
host_counts = pd.merge(host_counts, counts, left_on='host_species', right_on='host_species', how="right")
host_counts.replace(to_replace=[None], value="None", inplace=True)
# pd.unique(host_counts)
show(host_counts)
# df = px.data.tips()
# show(df)
print(list(host_counts.columns)[:-1])
fig = px.sunburst(host_counts, path=list(host_counts.columns)[:-1], values='count')
# fig.show()
fig.write_html("dataset_sanitise/galiez2017/taxonomy_fixing/server/vh_composition_sunburst.html")
fig = px.treemap(host_counts, path=[px.Constant("root"), *list(host_counts.columns)[:-1]], values='count')
fig.write_html("dataset_sanitise/galiez2017/taxonomy_fixing/server/vh_composition_treemap.html")

family_counts = df['host_family'].value_counts().reset_index()
family_counts_filtered = family_counts[family_counts['count'] > 10]
show(family_counts_filtered)
fig = px.bar(family_counts_filtered, x='host_family', y='count', text_auto=True)
fig.write_html("dataset_sanitise/galiez2017/taxonomy_fixing/server/vh_family_counts_hist.html")
# show(df)
# print(df)


['host_superkingdom', 'host_phylum', 'host_class', 'host_order', 'host_family', 'host_genus', 'host_species']


TypeError: treemap() got an unexpected keyword argument 'render_mode'

## get species count inside family

In [35]:
import pandas as pd
import numpy as np
import plotly.express as px
from itables import init_notebook_mode, show


df = pd.read_json("dataset_sanitise/galiez2017/taxonomy_fixing/server/virus.json", orient="index")
expanded_df = df['host'].apply(lambda x: pd.Series(x['lineage_names']))
expanded_df.columns = [f"host_{taxlevel}" for taxlevel in ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']]
df = pd.concat([df.drop('host', axis=1), expanded_df], axis=1)
#count occcurences in the host_family column
show(df)
counts = df['host_species'].value_counts().reset_index()
counts.columns = ['host_species', 'count']
show(counts)
host_counts = df[[*expanded_df.columns]].drop_duplicates()
host_counts = pd.merge(host_counts, counts, left_on='host_species', right_on='host_species', how="right")
host_counts.replace(to_replace=[None], value="None", inplace=True)
# pd.unique(host_counts)
show(host_counts)
species_count = host_counts['host_family'].value_counts().reset_index()
species_count.columns = ['host_family', 'count']
show(species_count)
print(species_count.loc[species_count['host_family'] == "Bacillaceae", 'count'].iloc[0])

9


## get unique family names to list

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from itables import init_notebook_mode, show
import glob
import pathlib as pl

# get unique family names to list

df_metadata = pd.read_json("dataset_sanitise/galiez2017/taxonomy_fixing/server/virus.json", orient="index")
# show(df_metadata['host'])
expanded_df = df_metadata['host'].apply(lambda x: pd.Series(x['lineage_names']))
expanded_df.columns = [f"host_{taxlevel}" for taxlevel in ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']]
df_metadata = pd.concat([df_metadata.drop('host', axis=1), expanded_df], axis=1)
show(df_metadata)
unique_family = df_metadata['host_family'].unique()
with open("dataset_sanitise/galiez2017/taxonomy_fixing/server/family_list.txt", "w") as f:
    f.write(",".join(unique_family))
print(unique_family)

## read all reports into df and visualise (facetted barplot)

In [41]:
import pandas as pd
import numpy as np
import plotly.express as px
from itables import init_notebook_mode, show
import glob
import pathlib as pl

# get unique family names to list

# df_metadata = pd.read_json("dataset_sanitise/galiez2017/taxonomy_fixing/server/virus.json", orient="index")
# # show(df_metadata['host'])
# expanded_df = df_metadata['host'].apply(lambda x: pd.Series(x['lineage_names']))
# expanded_df.columns = [f"host_{taxlevel}" for taxlevel in ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']]
# df_metadata = pd.concat([df_metadata.drop('host', axis=1), expanded_df], axis=1)
# show(df_metadata)
# unique_family = df_metadata['host_family'].unique()
# with open("dataset_sanitise/galiez2017/taxonomy_fixing/server/family_list.txt", "w") as f:
#     f.write(",".join(unique_family))
# print(unique_family)


# read all reports

# Specify the path
path = "misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports/*/Optimisation_report.xlsx"

# Find all xlsx files in the path
report_files = glob.glob(path)
print(report_files)

report_dfs = []
for file in report_files:
    taxname = pl.Path(file).parts[-2]
    report_df = pd.read_excel(file)
    report_df['taxname'] = f"{taxname} ({species_count.loc[species_count['host_family'] == taxname, 'count'].iloc[0]} unique species)"
    report_dfs.append(report_df)
combined_reports = pd.concat(report_dfs)

# report_dfs = [pd.read_excel(file) for file in report_files]
# combined_reports = pd.concat(report_dfs)

# # Add a new column indicating the file name
# combined_reports['source_file'] = [file for file in report_files for _ in range(len(pd.read_excel(file)))]

show(combined_reports)

# report_dfs = [pd.read_excel(file) for file in report_files]
# combined_reports = pd.concat(report_dfs)

# show(combined_reports)
fig = px.bar(combined_reports, x="Unnamed: 0", y=["top_species", "top3_species", "top_genus", "top3_genus"], barmode="group",
             category_orders={"taxname": combined_reports["taxname"].unique()},
             labels={"value": "Score", "variable": "Group"},
             facet_col="taxname", facet_col_wrap=6, facet_row_spacing=0.016, facet_col_spacing=0.02)  # Increase the facet_col_wrap value to make the facets bigger
fig.update_layout(
    showlegend=True
)
fig.write_html("misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports/optimisation_report.html")

# fig = px.bar(combined_reports, x="sex", y="total_bill", color="smoker", barmode="group",
#              facet_row="time", facet_col="day",
#              category_orders={"day": ["Thur", "Fri", "Sat", "Sun"],
#                               "time": ["Lunch", "Dinner"]})


['misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports\\Acholeplasmataceae\\Optimisation_report.xlsx', 'misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports\\Actinomycetaceae\\Optimisation_report.xlsx', 'misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports\\Aeromonadaceae\\Optimisation_report.xlsx', 'misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports\\Alcaligenaceae\\Optimisation_report.xlsx', 'misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports\\Alteromonadaceae\\Optimisation_report.xlsx', 'misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports\\Aurantimonadaceae\\Optimisation_report.xlsx', 'misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports\\Azospirillaceae\\Optimisation_report.xlsx', 'misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports\\Bacteroidaceae\\Optimisation_report.xlsx', 'misc/phastdna_galiez_species_filtertaxopt_e1_master/all_reports\\Brucellaceae\\Optimisation_report.xlsx', 'misc/ph

## get preds for each virus fragment (prep for vector creation for metaclassifier)

In [5]:
import pandas as pd
import pickle
from itables import init_notebook_mode, show

with open('misc/phastdna_galiez_taxfilter_pred-phastdna_galiez_species_filtertaxopt_e1_Bacillaceae_pickletest2/fastdna_pred_jobs', 'rb') as f:
    df_list = pickle.load(f)

# show(df_list[0])
df = df_list[0]
shapes = [df.shape for df in df_list]
nulls = [df.isnull().values.any() for df in df_list]
nunique_vals = [df.nunique() for df in df_list]
# apply unique to each column and if all are unique, then the make the column a list
# unique_cols = [col.tolist() if col.nunique() == col.shape[0] else col for col in df]

unique_vals = [df.apply(lambda x: pd.array(x.unique()).tolist() ) for df in df_list]
print(unique_vals[3])

Bacillus thuringiensis                    [0.0760252, 0.0749054, 0.0701305, 0.0718411, 0...
Bacillus cereus                           [0.0716631, 0.0705129, 0.0659114, 0.0675285, 0...
Bacillus anthracis                        [0.0710653, 0.0698306, 0.065018, 0.0666907, 0....
Bacillus bombysepticus                    [0.0710593, 0.069658, 0.0643828, 0.0661913, 0....
Bacillus mycoides                         [0.0708038, 0.0698645, 0.0658405, 0.0672835, 0...
Bacillus toyonensis                       [0.0646674, 0.0637139, 0.0599901, 0.0612883, 0...
Bacillus pseudomycoides                   [0.0640082, 0.0637496, 0.0619224, 0.0626739, 0...
Evansella cellulosilytica                 [0.0558294, 0.0557698, 0.0548365, 0.0552645, 0...
Bacillus cytotoxicus                      [0.0438727, 0.0438013, 0.0434394, 0.0435954, 0...
Lysinibacillus sphaericus                 [0.0433246, 0.0435338, 0.0439917, 0.0438918, 0...
Lysinibacillus varians                    [0.0432048, 0.0433916, 0.0438594, 0.04